In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm

uplifted = pd.read_csv('data/peaks.min_peak_score_0.6.thresh_0.5.csv')
uplifted.drop('Unnamed: 0', inplace=True, axis=1)
uplifted.tail()

,chrom,start,end,peak_pos,score
1037762,chr22,51240000,51240029,51240015,0.88
1037763,chr22,51240123,51240147,51240135,0.81
1037764,chr22,51240936,51240962,51240949,0.81
1037765,chr22,51242531,51242561,51242546,0.76
1037766,chr22,51243098,51243127,51243116,0.83


In [2]:
df = pd.read_csv('data/chr1.S15-30.filtered.csv', )
df.tail()

,Unnamed: 0,start,end,len_stem,len_loop
6743731,10941414,249239500.0,249239542.0,18.0,10.0
6743732,10941416,249239505.0,249239540.0,15.0,8.0
6743733,10941418,249239530.0,249239566.0,15.0,9.0
6743734,10941422,249239532.0,249239564.0,16.0,4.0
6743735,10941426,249239644.0,249239681.0,17.0,7.0


In [3]:
base_path = "/home/shared/STEMLOOPS/hg19/S16-50_L0-10_M3"
filename = "chr1.fa.S16-50_L0-10_M3.pal.cleaned"
path_to_file = os.path.join(base_path, filename)
with open(path_to_file , 'r') as f:
#     next(f)  # if there is description line
    splitted = f.read().split('\n')  # raw file separated by \n's

cols = ['start', 'end', 'len_stem', 'len_loop']#, 'seq_left', 'seq_right', 'full_seq', '1', '2', '3']
df = pd.DataFrame([sub.split("\t")[:4] for sub in tqdm(splitted)], columns=cols)  # TODO: full file (30 times bigger)
df = df.apply(pd.to_numeric)
df.drop(df.tail(1).index, inplace=True)
df['center'] = df[['start', 'end']].mean(axis=1).astype('int')

df = df.drop_duplicates(subset=['start']).drop_duplicates(['end'])
df.tail()


HBox(children=(IntProgress(value=0, max=100048), HTML(value='')))

,start,end,len_stem,len_loop,center
100042,249239154.0,249239198.0,18.0,10.0,249239176
100043,249239158.0,249239199.0,17.0,10.0,249239178
100044,249239183.0,249239227.0,18.0,10.0,249239205
100045,249239187.0,249239228.0,17.0,10.0,249239207
100046,249239305.0,249239345.0,16.0,9.0,249239325


### Hmm
Выделить все структуры, у которых основание ножки находятся на расстоянии от 0 до 10 нуклеотидов от границы нуклеосомы.

In [22]:
ss10bp = pd.read_csv('data/sec_struct_0-10bp_to_nucleosome.csv')1
ss10bp.tail()

,start,end,len_stem,len_loop,center
3453,46540271.0,46540305.0,18.0,1.0,46540288.0
3454,48949696.0,48949745.0,20.0,10.0,48949720.0
3455,48949697.0,48949742.0,19.0,8.0,48949719.0
3456,50160569.0,50160602.0,18.0,1.0,50160585.0
3457,50251558.0,50251593.0,16.0,4.0,50251575.0


Провести анализ, в какую область генома попали такие структуры - промотер, ген. Составить таблицу, в которой содержатся названия генов.


In [42]:
ptt = pd.read_csv('data/ptt_hg19.txt', delimiter='\t')
ptt1 = ptt[ptt['chrom'] == 'chr1']
ptt1.tail()

,#name,chrom,strand,txStart,txEnd,cdsStart,cdsEnd,exonCount,exonStarts,exonEnds,proteinID,alignID
7962,uc021pmg.1,chr1,+,249168446,249168518,249168446,249168446,1,"249168446,","249168518,",NaN,uc021pmg.1
7963,uc021pmh.1,chr1,+,249211536,249212562,249211536,249212562,1,"249211536,","249212562,",NP_001017434,uc021pmh.1
7964,uc001ifh.3,chr1,+,249200441,249213345,249208061,249212562,3,"249200441,249208014,249210800,","249200541,249208078,249213345,",Q6P3X8,uc001ifh.3
7965,uc001ifg.3,chr1,+,249200441,249213345,249211536,249212562,3,"249200441,249208014,249211477,","249200541,249208078,249213345,",NP_001017434,uc001ifg.3
7966,uc009xhd.3,chr1,+,249200441,249213345,249208750,249212562,4,"249200441,249208014,249208637,249210800,","249200541,249208078,249208758,249213345,",F5H4U7,uc009xhd.3


In [73]:

def make_10bp(df, uplifted):
    arr_4_rows = []
    for start_upl in tqdm(pd.to_numeric(uplifted.end)):
        temp_df = df[(df['txStart'] - 1000 < start_upl) & (df['txEnd'] > start_upl) & (df.strand == '+')]
        if not temp_df.empty:
            for ind, row in temp_df.iterrows():
                arr_4_rows.append(row)
    return pd.DataFrame(arr_4_rows)
relevant_ptt = make_10bp(ptt1, ss10bp)
relevant_ptt.tail()

HBox(children=(IntProgress(value=0, max=3458), HTML(value='')))

(3059, 12)

In [86]:
goa = pd.read_csv('data/goa_human.gaf', delimiter='\t', header=None)
goa.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
439605,UniProtKB,W5XKT8,SPACA6,NaN,GO:0016021,GO_REF:0000037,IEA,UniProtKB-KW:KW-0812,C,Sperm acrosome membrane-associated protein 6,SACA6_HUMAN|SPACA6|SPACA6P|UNQ2487/PRO5774,protein,taxon:9606,20171216,UniProt,NaN,NaN
439606,UniProtKB,X5D2U9,HLA-DRB4,NaN,GO:0002504,GO_REF:0000038,IEA,UniProtKB-KW:KW-0491,P,"HLA class II histocompatibility antigen, DR be...",X5D2U9_HUMAN|HLA-DRB4,protein,taxon:9606,20171216,UniProt,NaN,NaN
439607,UniProtKB,X5D2U9,HLA-DRB4,NaN,GO:0006955,GO_REF:0000002,IEA,InterPro:IPR000353|InterPro:IPR014745,P,"HLA class II histocompatibility antigen, DR be...",X5D2U9_HUMAN|HLA-DRB4,protein,taxon:9606,20171216,InterPro,NaN,NaN
439608,UniProtKB,X5D2U9,HLA-DRB4,NaN,GO:0016021,GO_REF:0000038,IEA,UniProtKB-KW:KW-0812,C,"HLA class II histocompatibility antigen, DR be...",X5D2U9_HUMAN|HLA-DRB4,protein,taxon:9606,20171216,UniProt,NaN,NaN
439609,UniProtKB,X5D2U9,HLA-DRB4,NaN,GO:0042613,GO_REF:0000038,IEA,UniProtKB-KW:KW-0491,C,"HLA class II histocompatibility antigen, DR be...",X5D2U9_HUMAN|HLA-DRB4,protein,taxon:9606,20171216,UniProt,NaN,NaN


In [104]:
relevant_goa = goa[goa[1].isin(relevant_ptt.proteinID)].drop_duplicates(1)
relevant_goa[9].to_csv('data/relevant_goa_names.csv')